In [3]:
!pip install opencv_contrib_python
!pip install numpy

In [ ]:
#download the models from the link below and save them in the models folder.
#download the black and white pictures in the folder. In my case the name of folder is is old
#https://eecs.berkeley.edu/~rich.zhang...
#https://github.com/richzhang/colorization/blob/caffe/models/colorization_deploy_v2.prototxt
#https://github.com/HemantKArya/BW2Color/blob/master/model/pts_in_hull.npy

import numpy as np 
import cv2

print("loading models.....")
net = cv2.dnn.readNetFromCaffe('C:/Users/Nasik/Desktop/bnw_to_col/models/colorization_deploy_v2.prototxt','C:/Users/Nasik/Desktop/bnw_to_col/models/colorization_release_v2.caffemodel')
pts = np.load('C:/Users/Nasik/Desktop/bnw_to_col/models/pts_in_hull.npy')


class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2,313,1,1)

net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1,313],2.606,dtype='float32')]


image = cv2.imread('C:/Users/Nasik/Desktop/bnw_to_col/old/Image0115.jpg')
scaled = image.astype("float32")/255.0
lab = cv2.cvtColor(scaled,cv2.COLOR_BGR2LAB)


resized = cv2.resize(lab,(224,224))
L = cv2.split(resized)[0]
L -= 50


net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1,2,0))

ab = cv2.resize(ab, (image.shape[1],image.shape[0]))

L = cv2.split(lab)[0]
colorized = np.concatenate((L[:,:,np.newaxis], ab), axis=2)

colorized = cv2.cvtColor(colorized,cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized,0,1)

colorized = (255 * colorized).astype("uint8")

cv2.imshow("Original",image)
cv2.imshow("Colorized",colorized)

cv2.waitKey(0)

loading models.....
